# Установка модулей, импорт библиотек

In [ ]:
!pip install sacremoses

In [ ]:
!pip install nlpaug

In [4]:
!pip3 install tensorflow_text>=2.0.0rc0

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [10]:
# import nlpaug.augmenter.word as naw
import pandas as pd
from collections import Counter
from transformers import AutoTokenizer, AutoModel
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import random
import time

# **Оверсэмплинг.**
Создание новых примеров/датасетов

In [7]:
# Создаем аугментер для замены синонимов
aug_syn = naw.SynonymAug(aug_src='wordnet')

In [13]:
aug_syn.augment("This is a sample text for augmentation.")

['This exist a sample text edition for augmentation.']

In [11]:
df = pd.read_csv('/content/drive/MyDrive/cw_classifier/tripadvisor_hotel_reviews.csv')

In [23]:
sen4 = df[df['Rating'] == 4]['Review'].to_list()
sen3 = df[df['Rating'] == 3]['Review'].to_list()
sen2 = df[df['Rating'] == 2]['Review'].to_list()
sen1 = df[df['Rating'] == 1]['Review'].to_list()

sen4_syn = []
for text in sen4:
  sen4_syn += aug_syn.augment(text)
sen3_syn = []
for text in sen3:
  sen3_syn += aug_syn.augment(text)
  sen3_syn += aug_syn.augment(text)
  sen3_syn += aug_syn.augment(text)
sen2_syn = []
for text in sen2:
  sen2_syn += aug_syn.augment(text)
  sen2_syn += aug_syn.augment(text)
  sen2_syn += aug_syn.augment(text)
  sen2_syn += aug_syn.augment(text)
sen1_syn = []
for text in sen1:
  sen1_syn += aug_syn.augment(text)
  sen1_syn += aug_syn.augment(text)
  sen1_syn += aug_syn.augment(text)
  sen1_syn += aug_syn.augment(text)
  sen1_syn += aug_syn.augment(text)

df4 = pd.DataFrame({'Review': sen4_syn[:3000], 'Rating': [4]*len(sen4_syn[:3000])})
df3 = pd.DataFrame({'Review': sen3_syn, 'Rating': [3]*len(sen3_syn)})
df2 = pd.DataFrame({'Review': sen2_syn, 'Rating': [2]*len(sen2_syn)})
df1 = pd.DataFrame({'Review': sen1_syn, 'Rating': [1]*len(sen1_syn)})

data = pd.concat([df, df1, df2, df3, df4])

In [24]:
Counter(data['Rating'])

Counter({4: 9039, 2: 8965, 3: 8736, 5: 9054, 1: 8526})

In [25]:
data.to_csv('sent_balanced_by_syn.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [46]:
def generate_text(input_text, model, tokenizer):
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  outputs = model.generate(input_ids, max_length=40, do_sample=True, temperature=0.6, top_k=30)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return generated_text

In [54]:
def new_sent_T5(lst):
  sents = []
  for text in lst:
    words = text.split()
    input_text = ' '.join(words[:int(len(words)*0.2)])
    sents.append(generate_text(input_text, model, tokenizer))
  return sents

In [53]:
words = data['Review'].iloc[0].split()
input_text = ' '.join(words[:int(len(words)*0.2)])
generate_text(input_text, model, tokenizer)

'hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews, got good deal stay hotel anniversary, checked out late night took good deal stay hotel anniversary, took good'

In [55]:
new_sent_T5(sen1[:10])

['4th 2007, was in monaco feb 4th 2007 3 valet young man, pulled valet,, pulled valet, waited',
 'arranged stay warwick seattle hotel disapointment trip, arranged stay seattle hotel disapointment trip, arranged stay seattle hotel disapointment trip,',
 'bad choice, king bed room away elevator/ice. the room was not available for breakfast. room king bed room away elevator/ice.no elevator/ice',
 'the the place, inn terrible, inn horrible,. the inn terrible, horrible, terrible, i hated inn terrible,i',
 'grunge grunge grunge ace grunge ace grunge grunge grunge ace grunge grunge ace grung',
 'thursday internet reservation rang hour later add thursday booking, internet reservation friday rang hour later add thursday booking, internet reservation friday rang',
 'like hospital rooms single rooms like hospital rooms single rooms like hospital rooms like single rooms like hospital rooms like hospital rooms like single rooms like hospital rooms like single rooms like hospital rooms like hospital

# Вычисление эмбеддингов LaBASE

In [7]:
data = pd.read_csv('/content/drive/MyDrive/cw_classifier/Hotel_EN_balanced_by_syn.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
data

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
44315,great location small rooms good note value ove...,4
44316,family birthday family brother 50th natal day ...,4
44317,splendid time value took husband fiftieth birt...,4
44318,friendly good fix stayed 3 nights 16th / 19th ...,4


In [14]:
44320/2/2/2/2/2

1385.0

In [ ]:
data['Rating'].hist()

In [72]:
Counter(data['Rating'])

Counter({4: 9039, 2: 8965, 3: 8736, 5: 9054, 1: 8526})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

In [16]:
sentences = data['Review'].to_list()
total_embed = []
for i in range(1, 33):
  encoded_input = tokenizer(sentences[1385*(i-1):1385*i], padding=True, truncation=True, max_length=64, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  embeddings = model_output.pooler_output
  embeddings = torch.nn.functional.normalize(embeddings)
  total_embed += embeddings.tolist()

In [17]:
data['LaBASE'] = total_embed

In [19]:
data

,Review,Rating,LaBASE
0,nice hotel expensive parking got good deal sta...,4,"[-0.04870546609163284, 0.03937816992402077, -0..."
1,ok nothing special charge diamond member hilto...,2,"[0.010228053666651249, -0.002325975801795721, ..."
2,nice rooms not 4* experience hotel monaco seat...,3,"[-0.005978066008538008, 0.031884144991636276, ..."
3,"unique, great stay, wonderful time hotel monac...",5,"[-0.004608594346791506, 0.059613071382045746, ..."
4,"great stay great stay, went seahawk game aweso...",5,"[-0.058320462703704834, 0.04319010674953461, -..."
...,...,...,...
44315,great location small rooms good note value ove...,4,"[-0.04568036273121834, 0.030866196379065514, -..."
44316,family birthday family brother 50th natal day ...,4,"[-0.0034492984414100647, 0.05806007608771324, ..."
44317,splendid time value took husband fiftieth birt...,4,"[-0.004192113410681486, 0.051920898258686066, ..."
44318,friendly good fix stayed 3 nights 16th / 19th ...,4,"[-0.026278335601091385, 0.04283636063337326, -..."


In [18]:
data.to_csv('12_hours.csv')

In [1]:
import pandas as pd

In [8]:
df = pd.read_json('/content/drive/MyDrive/cw_classifier/parsed.jl', lines=True)
df['rating'] = df['rating']/10
df.drop(['hotel', 'address', 'date'], axis=1, inplace=True)
df.rename(columns={'content': 'review'}, inplace=True)

grouped_df = df.groupby('rating')
dfs_by_rating = [grouped_df.get_group(x) for x in grouped_df.groups]
min_count = min([len(d) for d in dfs_by_rating])
dfs_sampled = [d.sample(min_count, random_state=42) for d in dfs_by_rating]
df_balanced = pd.concat(dfs_sampled)

In [11]:
sentences = df_balanced['review'].to_list()
total_embed = []
start_time = time.time()
for i in range(0, len(sentences), 1000):
  if i % 2000 == 0:
    print('i =', i, '\ntime:', time.time() - start_time, '\n')
  if i+1000 > len(sentences):
    encoded_input = tokenizer(sentences[i:], padding=True, truncation=True, max_length=64, return_tensors='pt')
  else:
    encoded_input = tokenizer(sentences[i:i+1000], padding=True, truncation=True, max_length=64, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  embeddings = model_output.pooler_output
  embeddings = torch.nn.functional.normalize(embeddings)
  total_embed += embeddings.tolist()
df_balanced['bert-tiny'] = total_embed

i = 0 
time: 0.00030112266540527344 

i = 2000 
time: 470.4292974472046 

i = 4000 
time: 926.7644488811493 

i = 6000 
time: 1360.6381995677948 

i = 8000 
time: 1800.22118973732 

i = 10000 
time: 2236.0386958122253 

i = 12000 
time: 2669.9680726528168 

i = 14000 
time: 3103.285476922989 

i = 16000 
time: 3536.550973176956 

i = 18000 
time: 3968.3824253082275 

i = 20000 
time: 4399.591522693634 

i = 22000 
time: 4832.1450917720795 

i = 24000 
time: 5264.638788700104 

i = 26000 
time: 5697.036624908447 

i = 28000 
time: 6128.70223236084 

i = 30000 
time: 6558.103777170181 

i = 32000 
time: 6988.943466186523 

i = 34000 
time: 7420.428470611572 



In [12]:
df.rename(columns={'bert-tiny': 'LaBASE'}, inplace=True)

In [13]:
df_balanced.to_csv('/content/drive/MyDrive/cw_classifier/sec_balanced_hotel_reviews.csv')